In [1]:
%run ../Python_files/util_data_storage_and_load.py

In [2]:
%run ../Python_files/load_dicts.py

In [3]:
%run ../Python_files/util.py

In [4]:
# load logit_route_choice_probability_matrix
P = zload('../temp_files/logit_route_choice_probability_matrix.pkz')

In [5]:
import numpy as np
from numpy.linalg import inv

P = np.matrix(P)

In [6]:
np.size(P, 0), np.size(P, 1)

(56, 140)

In [7]:
import numpy as np
from numpy.linalg import inv

def samp_cov(x):
    """
    x: sample matrix, each column is a link flow vector sample; 24 * K
    K: number of samples
    S: sample covariance matrix
    ----------------
    return: inv(S)
    ----------------
    """
    x = np.matrix(x)
    K = np.size(x, 1)
    x_mean = sum(x[:,k] for k in range(K)) / K
    S = sum(np.dot(x[:,k] - x_mean, np.transpose(x[:,k] - x_mean)) for k in range(K)) / (K - 1)
    return S

In [8]:
# load path-link incidence matrix
A = zload('../temp_files/path-link_incidence_matrix.pkz')

In [9]:
# load link counts data

import json

with open('../temp_files/link_day_minute_Apr_dict_JSON.json', 'r') as json_file:
    link_day_minute_Apr_dict_JSON = json.load(json_file)

In [10]:
week_day_Apr_list = [2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 16, 17, 18, 19, 20, 23, 24, 25, 26, 27, 30]

In [11]:
link_day_minute_Apr_list = []
for link_idx in range(24):
    for day in week_day_Apr_list: 
        for minute_idx in range(120):
            key = 'link_' + str(link_idx) + '_' + str(day)
            link_day_minute_Apr_list.append(link_day_minute_Apr_dict_JSON[key] ['NT_flow_minute'][minute_idx])

In [12]:
len(link_day_minute_Apr_list)

60480

In [13]:
x = np.matrix(link_day_minute_Apr_list)
x = np.matrix.reshape(x, 24, 2520)

x = np.nan_to_num(x)
y = np.array(np.transpose(x))
y = y[np.all(y != 0, axis=1)]
x = np.transpose(y)
x = np.matrix(x)

In [14]:
np.size(x, 0), np.size(x, 1)

(24, 2516)

In [15]:
x[:,:2]

matrix([[  975144.75991934,   957700.5924282 ],
        [  993674.68453025,  1006905.34294973],
        [ 1848348.05701542,  1834395.1268862 ],
        [ 1929351.11768772,  1956748.59334819],
        [  480903.1784188 ,   486205.56048652],
        [  601926.70221072,   596920.4623292 ],
        [ 1178769.64348927,  1190911.93260812],
        [ 1171879.9565668 ,  1159952.18650977],
        [ 1101114.27310236,  1086938.86206366],
        [ 1153761.29440029,  1127876.54602288],
        [ 1138120.38469258,  1339113.05449946],
        [ 1010641.91315051,  1107218.98786112],
        [  276198.5322158 ,   273056.24422526],
        [  303128.33286372,   296010.32957641],
        [  455406.87899511,   462952.004351  ],
        [  421616.03522462,   373940.22154089],
        [  612150.95784024,   622987.46178501],
        [  601824.61600099,   607781.95081361],
        [ 1015283.36056256,  1008785.36462254],
        [  915815.56318712,   900562.98509864],
        [ 1510876.97196134,  1504511.296

In [16]:
np.size(A,0), np.size(A,1)

(24, 140)

In [17]:
from gurobipy import *

L = 56  # dimension of lam

K = np.size(x, 1)
S = samp_cov(x)
inv_S = inv(S)

A_t = np.transpose(A)
P_t = np.transpose(P)
# PA'
PA_t = np.dot(P, A_t)
# AP_t
AP_t = np.transpose(PA_t)

Q = np.dot(np.dot(PA_t, inv_S), AP_t)
b = sum([np.dot(np.dot(PA_t, inv_S), x[:, k]) for k in range(K)])


model = Model("OD_matrix_estimation")

lam = []
for l in range(L):
    lam.append(model.addVar(name='lam_' + str(l)))

model.update() 

# Set objective: (K/2) lam' * Q * lam - b * lam
obj = 0
for i in range(L):
    for j in range(L):
        obj += (1.0 /2) * K * lam[i] * Q[i, j] * lam[j]
for l in range(L):
    obj += - b[l] * lam[l]
model.setObjective(obj)

# Add constraint: lam >= 0
for l in range(L):
    model.addConstr(lam[l] >= 0)

model.update() 

model.optimize()

lam_list = []
for v in model.getVars():
    print('%s %g' % (v.varName, v.x))
    lam_list.append(v.x)
print('Obj: %g' % obj.getValue())

Optimize a model with 56 rows, 56 columns and 56 nonzeros
Model has 1596 quadratic objective terms
Coefficient statistics:
  Matrix range    [1e+00, 1e+00]
  Objective range [9e-02, 1e+01]
  Bounds range    [0e+00, 0e+00]
  RHS range       [0e+00, 0e+00]
Presolve removed 56 rows and 0 columns
Presolve time: 0.00s
Presolved: 0 rows, 56 columns, 0 nonzeros
Presolved model has 1596 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 24
 AA' NZ     : 2.760e+02
 Factor NZ  : 3.000e+02
 Factor Ops : 4.900e+03 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0  -2.63272261e+07  0.00000000e+00  5.71e+03 2.15e+03  1.00e+06     0s
   1  -2.17439487e+07 -1.00770122e+07  1.40e+03 5.28e+02  2.88e+05     0s
   2  -1.78458064e+07 -1.96269412e+07  6.13e+01 2.31e+01  5.65e+04     0s
   3  -1.80416550e+07 -1.82906891e+07  6.13e-05 2.31e-05  4.4

In [18]:
# write estimation result to file
n = 8  # number of nodes
with open('../temp_files/OD_demand_matrix_Apr_weekday_NT.txt', 'w') as the_file:
    idx = 0
    for i in range(n + 1)[1:]:
        for j in range(n + 1)[1:]:
            if i != j: 
                the_file.write("%d,%d,%f\n" %(i, j, lam_list[idx]))
                idx += 1

In [19]:
lam_list

[1.586647251604345e-08,
 590886.2532755232,
 439344.417101184,
 61958.474075583035,
 194476.72745318778,
 4.356644703580083e-05,
 0.00021576715281983877,
 443562.8419036132,
 9.816179744411704e-10,
 317812.05320098175,
 9.81611065906092e-10,
 9.816163402122508e-10,
 9.81599117577414e-10,
 9.882042578756148e-10,
 12622.217522304552,
 19218.133134649008,
 8.122645941727354e-07,
 79782.17554085063,
 485018.0299282034,
 1028131.4040547326,
 0.00010910066142823596,
 1.4767649928431985e-05,
 891969.2086208732,
 2.173465949722336e-06,
 125386.07970544738,
 39860.72641646169,
 2.366584221008333e-06,
 410836.4084035846,
 941857.0217666934,
 83800.17369650022,
 1.9404834784024125e-06,
 156385.50391313856,
 98233.1586309046,
 5.883051970839292e-07,
 5.566403481352096e-06,
 709716.0988062294,
 72736.14420605393,
 1.971923689512978e-06,
 0.00011885435947559322,
 369394.0698624354,
 637895.0111150197,
 32888.2557106826,
 8.087080739165291e-06,
 5.0181225135002925e-06,
 2.042763592166553e-06,
 3.2398